In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import eli5
from eli5.sklearn import PermutationImportance

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras import layers
from keras.layers import Dense, BatchNormalization, Flatten, LSTM, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.utils import to_categorical

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'sample_submission.csv']


/opt/conda/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


In [2]:
# define base model
def baseline_model():
    # create model
#     model = Sequential()
#     model.add(Dense(128, input_dim=40, kernel_initializer='normal', activation='relu'))
#     model.add(layers.Dense(256, kernel_initializer='normal', activation='relu'))
#     model.add(layers.Dense(128, kernel_initializer='normal', activation='relu'))

#     model.add(Dense(1, kernel_initializer='normal'))
#     # Compile model
#     model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
    
    # define model
#     model = Sequential()
#     model.add(LSTM(128, input_shape=(None, 40)))
#     model.add(Dense(1, activation='linear'))
#     # compile model
#     model.compile(loss='mae', optimizer='adam')

#     model = Sequential()
#     model.add(LSTM(4, input_shape=(1, 1)))
#     model.add(Dense(1))
#     model.compile(loss='mean_absolute_error', optimizer='adam')
    
    
    model = Sequential()

    model.add(LSTM(128, input_shape=((1, 40)), activation='linear', return_sequences=True))
    model.add(Dropout(0.1))

    model.add(LSTM(256, activation='linear'))
    model.add(Dropout(0.2))


    model.add(Dense(256, activation='linear'))
#     model.add(Dropout(0.2))
    
    model.add(Dense(128, activation='linear'))
#     model.add(Dropout(0.2))

    model.add(Dense(64, activation='linear'))
#     model.add(Dropout(0.2))
 
    model.add(Dense(32, activation='linear'))
#     model.add(Dropout(0.2))
    
    
    model.add(Dense(1, activation='linear'))

    model.compile(loss='mae', optimizer='adam', metrics=['mean_absolute_error'])


    return model

In [3]:


# Path of the file to read. We changed the directory structure to simplify submitting to a competition
iowa_file_path = '../input/train.csv'

home_data = pd.read_csv(iowa_file_path)

# Create target object and call it y
y = home_data.SalePrice

# Create X
X = home_data.drop(['SalePrice'], axis=1)
X = X.select_dtypes(exclude=['object'])

# Split into validation and training data
# train_X, test_X, train_y, test_y= train_test_split(X,y,
#                                                     train_size=0.7, 
#                                                     test_size=0.3, 
#                                                     random_state=0)

cols_with_missing = (col for col in X.columns if X[col].isnull().any())

for col in cols_with_missing:
    X[col + '_was_missing'] = X[col].isnull()
#     test_X[col + '_was_missing'] = test_X[col].isnull()

# X = X.drop(columns=cols_with_missing)
# Imputation
features = X.columns
my_imputer = SimpleImputer()
X = my_imputer.fit_transform(X)
# train_X = my_imputer.fit_transform(train_X)
# test_X = my_imputer.transform(test_X)
# X = np.concatenate((train_X,test_X))
# y = np.concatenate((train_y, test_y))
min_max_scaler = MinMaxScaler()
standard = StandardScaler()
# min_max_scaler = standard
X = min_max_scaler.fit_transform(X)
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

print((X.shape[0], X.shape[1]))
print(X.shape)
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
# estimator = KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=-2)

# kfold = KFold(n_splits=10, random_state=seed)
# results = cross_val_score(estimator, X, y, cv=kfold)
# print("Results: %.2f (%.2f) MAE" % (results.mean(), results.std()))
# Key_Features = pd.DataFrame({'feature': list(features),
#                    'importance': estimator.feature_importances_}).\
#                     sort_values('importance', ascending = False)

estimators = []
# estimators.append(('standardize', StandardScaler()))


estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=80, batch_size=32, verbose=0)))
pipeline = Pipeline(estimators)
# kfold = KFold(n_splits=4, random_state=seed)
# results = cross_val_score(pipeline, X, y, cv=kfold)
# print("Standardized: %.2f (%.2f) MAE" % (results.mean(), results.std()))


(1460, 1)
(1460, 1, 40)


In [4]:
# all 128 batch sizes
# lstm 256 (0.2) --> lstm 128 (0.2) -> dense 128 (0.2) -> dense 128 (0.2) -> dense 64 (0.2) [80, 128]:                    -20103.75 (1845.26) MAE
# lstm 256 (0.2) --> lstm 128 (0.2) -> dense 128 (0.2) -> dense 128 (0.2) -> dense 64 (0.2) [70, 128]:                    -20318.08 (1596.56) MAE
# lstm 256 (0.2) --> lstm 256 (0.2) -> dense 128 (0.2) -> dense 128 (0.2) -> dense 64 (0.2) [70, 128]:                    -20275.79 (1662.84) MAE
# lstm 256 (0.2) --> lstm 256 (0.2) -> dense 128 (0.2) -> dense 128 (0.2) -> dense 64 (0.2) [80, 128]:                    -19831.02 (1896.39) MAE
# lstm 512 (0.2) --> lstm 256 (0.2) -> dense 128 (0.2) -> dense 128 (0.2) -> dense 64 (0.2) [80, 128]:                    -19844.62 (1773.10) MAE
# lstm 128 (0.2) --> lstm 256 (0.2) -> dense 128 (0.2) -> dense 128 (0.2) -> dense 64 (0.2) [80, 128]:                    -20227.35 (1658.87) MAE
# lstm 128 (0.2) --> lstm 256 (0.2) -> dense 128 (0.2) -> dense 128 (0.2) -> dense 64 (0.2) [100, 128]:                   -19564.36 (1721.07) MAE
# lstm 128 (0.2) --> lstm 64 (0.2) -> dense 128 (0.2) -> dense 100 (0.2) -> dense 64 (0.2) -> dense 32 (0.2) [100, 128]:  -20192.38 (2503.99) MAE
# lstm 128 (0.2) --> lstm 256 (0.2) -> dense 128 (0.2) -> dense 100 (0.2) -> dense 64 (0.2) -> dense 32 (0.2) [100, 128]: -19448.54 (1978.56) MAE
# lstm 256 (0.2) --> lstm 256 (0.2) -> dense 128 (0.2) -> dense 100 (0.2) -> dense 64 (0.2) -> dense 32 (0.2) [100, 128]: -19745.67 (2118.28) MAE
# lstm 256 (0.2) --> lstm 128 (0.2) -> dense 128 (0.2) -> dense 100 (0.2) -> dense 64 (0.2) -> dense 32 (0.2) [100, 128]: -19728.36 (1460.42) MAE
# lstm 128 (0.2) --> lstm 256 (0.2) -> dense 128 (0.2) -> dense 100 (0.2) -> dense 64 (0.2) -> dense 32 (0.2) [80, 128]:  -20270.39 (1886.73) MAE
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 128 (0.2) -> dense 100 (0.2) -> dense 64 (0.2) -> dense 32 (0.2) [100, 128]: -19352.87 (1786.66) MAE
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 128 (0.2) -> dense 128 (0.2) -> dense 64 (0.2) -> dense 32 (0.2) [100, 128]: -19368.60 (1775.76) MAE
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 256 (0.2) -> dense 128 (0.2) -> dense 64 (0.2) -> dense 32 (0.2) [100, 128]: -19318.86 (2048.19) MAE
# softmax activation
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 256 (0.2) -> dense 128 (0.2) -> dense 64 (0.2) -> dense 32 (0.2) [100, 128]: -180918.88 (2514.87) MAE
# all relu
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 256 (0.2) -> dense 128 (0.2) -> dense 64 (0.2) -> dense 32 (0.2) [100, 128]: -19214.07 (1918.02) MAE
# all linear
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 256 (0.2) -> dense 128 (0.2) -> dense 64 (0.2) -> dense 32 (0.2) [100, 128]: -19615.55 (1522.68) MAE
# standard scaler (all linear)
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 256 (0.2) -> dense 128 (0.2) -> dense 64 (0.2) -> dense 32 (0.2) [100, 128]:  -18246.12 (1304.44) MAE
# standard scaler (all relu)
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 256 (0.2) -> dense 128 (0.2) -> dense 64 (0.2) -> dense 32 (0.2) [100, 128]: -20008.41 (1320.68) MAE
# standard scaler (normal activations)
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 256 (0.2) -> dense 128 (0.2) -> dense 64 (0.2) -> dense 32 (0.2) [100, 128]: -19576.05 (1553.72) MAE
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 256 -> dense 128 -> dense 64 -> dense 32 [100, 128]:                         -18590.97 (1596.56) MAE                     
# standard scaler (all linear)
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 256 -> dense 128 -> dense 64 -> dense 32 [100, 128]:                         -17980.04 (1265.17) MAE
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 256 -> dense 128 -> dense 64 -> dense 32 [80, 128]:                          -17597.71 (998.45) MAE
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 256 -> dense 128 -> dense 64 -> dense 32 [70, 128]:                          -18737.23 (1205.90) MAE
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 256 -> dense 128 -> dense 64 -> dense 32 [75, 128]:                          -17778.31 (1084.07) MAE
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 256 -> dense 128 -> dense 64 -> dense 32 [85, 128]:                          -17849.28 (1375.23) MAE
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 256 -> dense 128 -> dense 64 -> dense 32 [150, 128]:                         -17820.91 (1705.90) MAE
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 256 -> dense 128 -> dense 64 -> dense 32 [80, 64]:                           -17172.10 (1481.64) MAE
# not imputing, dropping columns with nulls
# lstm 128 (0.1) --> lstm 256 (0.2) -> dense 256 -> dense 128 -> dense 64 -> dense 32 [80, 128]:                          -17016.59 (1400.03) MAE
                     

# pca = PCA(n_components=40)
# pca.fit(X)

# plt.plot(np.cumsum(pca.explained_variance_ratio_))
# plt.xlabel('Number of components')
# plt.ylabel('Cumulative explained variance')

In [5]:
print(1)
rf_model_on_full_data = pipeline
print(2)

# cols_with_missing = [col for col in X.columns 
#                                  if X[col].isnull().any()]
# X = X.drop(cols_with_missing, axis=1)

# fit rf_model_on_full_data on all data from the 
rf_model_on_full_data.fit(X, y)
print(3)

# path to file you will use for predictions
test_data_path = '../input/test.csv'

# read test data file using pandas
test_data = pd.read_csv(test_data_path)

cols_with_missing = (col for col in test_data.columns if test_data[col].isnull().any())

for col in cols_with_missing:
    test_data[col + '_was_missing'] = test_data[col].isnull()

# create test_X which comes from test_data but includes only the columns you used for prediction.
# The list of columns is stored in a variable called features
test_X = test_data[features]


# my_imputer = SimpleImputer()
test_X = my_imputer.transform(test_X)

print(4)
test_X = min_max_scaler.transform(test_X)

test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))


# make predictions which we will submit. 
preds = rf_model_on_full_data.predict(test_X)
print(5)

# The lines below shows you how to save your data in the format needed to score it in the competition
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': preds})

output.to_csv('submission.csv', index=False)


1
2
3
4
5


In [6]:
# C_mat = home_data.drop(columns=['Id']).corr()
# fig = plt.figure(figsize = (20,1))
# sales_corr = pd.DataFrame()
# for name in C_mat.columns:
#     to_add = np.absolute([C_mat[name][36]])
#     if to_add[0] > 0.3 and name != 'SalePrice':
#         sales_corr[name] = [C_mat[name][36]]
# print(sales_corr.columns)

# sb.heatmap(sales_corr, square = True, cmap="YlGnBu")
